# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

###  Norvig's solution

In [1]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [2]:
def spelling_corrector_norvig(sentence):
    sentence_arr = sentence.lower().split()
    new_sentence = []
    for word in sentence_arr:
        new_sentence.append(correction(word))
    return ' '.join(new_sentence)

### My solution

In [3]:
bigram_dict = {} # {(first_word, second_word): frequency, ...}
vocab_check = [] # [(first_word, ' '), (' ', second_word), ...]

In [4]:
with open('bigrams.txt', 'r', encoding='latin-1') as file:
    for line in file:
        line = line[:-1].lower().split('\t')
        bigram_dict[(line[1], line[2])] = int(line[0])
        vocab_check.append((line[1], ' '))
        vocab_check.append((' ', line[2]))

In [5]:
vocab_check_dict = Counter(vocab_check)

In [6]:
# The function uses the 'know()', 'edits1()' and 'edits2()' functions from Norvig's solution to optimize time
def candidates_word(word, mode=1):
    """
    Generate possible spelling corrections for a words based on the entered word and mode.

    Parameters:
        word (str): The input word.
        mode (int): The mode for generating candidates.
                    1 - Generate candidates with one correction.
                    2 - Generate candidates with two corrections.

    Returns:
        list: List of candidate words.
    """

    candidates = list(known(edits1(word)))
    if mode == 2:
        candidates.extend(list(known(edits2(word))))
    if known([word]):
        # If such a word exists, add it
        candidates.append(word)
    if not candidates and mode == 2:
        # If no candidates found and mode is 2, return the original word itself
        return [word]
    return candidates

In [7]:
#  takes into account the position of the word
def variants_word_calculate(variants, place):
    """
    Filter variants based on their position in the vocabulary.

    Parameters:
        variants (list): List of word variants.
        place (int): Position indicator.
                     1 - Variant appears as the first word.
                     2 - Variant appears as the second word.

    Returns:
        list: Filtered list of word variants based on their position in the vocabulary.
    """
    out = []
    for variant in variants:
        if place == 1:
            if (variant, ' ') in vocab_check:
                out.append(variant)
        else:
            if (' ', variant) in vocab_check:
                out.append(variant)
    return out

In [8]:
def variants_word(word, place, mode=1):
    """
    Generate word variants based on the input word and its position in the context.

    Parameters:
        word (str): The input word.
        place (int): Position indicator.
                     1 - Variant appears as the first word.
                     2 - Variant appears as the second word.
        mode (int): Mode for candidate generation.
                    1 - Generate candidates with one correction.
                    2 - Generate candidates with two corrections.

    Returns:
        list: List of word variants.
    """
    out = []
    variants = candidates_word(word) # looking for candidates with one correction
    out = variants_word_calculate(variants, place) # checking that words can stand in a given place
    if len(out) == 0:
      # if there are no such words, then look for words with two corrections
      variants = candidates_word(word, 2)
      out = variants_word_calculate(variants, place)
      if len(out) == 0:
        # if there are no such words, we return the selected variants
        out = variants
    return out

In [9]:
# even if a word exists, I find possible words to replace it
def alternatives_word(word, place):
    """
    Generate alternative words based on the input word and its position in the context.

    Parameters:
        word (str): The input word.
        place (int): Position indicator.
                     1 - Word appears as the first word.
                     2 - Word appears as the second word.

    Returns:
        list: List of alternative words.
    """
    # Determine the threshold based on the position of the word in the vocabulary
    if place == 1:
      threshold = vocab_check_dict[(word, ' ')]
    else:
      threshold = vocab_check_dict[(' ', word)]
    out = [word] # writing down the base word as it exists
    alternatives = candidates_word(word)
    for alternative in alternatives:
        if place == 1:
            if (alternative, ' ') in vocab_check:
                if vocab_check_dict[(alternative, ' ')] > threshold:
                    # checking that the alternative word is more common
                    out.append(alternative)
        else:
            if (' ', alternative) in vocab_check:
                if vocab_check_dict[(' ', alternative)] > threshold:
                    out.append(alternative)
    return out

In [10]:
def calculate(current_word, next_word, best_word, previous_word=None):
    """
    Calculate how often the word is used in a given context

    Parameters:
        current_word (list): List of current word candidates.
        next_word (list): List of next word candidates.
        best_word (str): The best candidate word according to Norvig's solution.
        previous_word (str): The previous word in the sequence. Default is None.

    Returns:
        list: List of tuples containing the candidate word, its cost, and possible next words.
    """
    candidates = []
    for f_word in current_word:
        cost = 0
        next_possible_words = []
        if f_word == best_word:
            # Add cost for the best word
            cost += 1000
        # Calculate cost based on bigram probabilities
        for s_word in next_word:
            if (f_word, s_word) in bigram_dict:
                cost += bigram_dict[(f_word, s_word)]
                next_possible_words.append(s_word) # add next possible word
        # If no valid next word found, consider all next words
        if len(next_possible_words) == 0:
            next_possible_words.extend(next_word)
        # Add cost based on previous word
        if previous_word is not None:
            if (previous_word, f_word) in bigram_dict:
                cost += bigram_dict[(previous_word, f_word)] + 1000
        # Append candidate with its cost and possible next words
        candidates.append((f_word, cost, next_possible_words))
    return candidates

In [11]:
def find_word(current_word, next_word, best_word, previous_word=None):
    """
    Find the word with the highest score among the candidates.

    Parameters:
        current_word (list): List of current word candidates.
        next_word (list): List of next word candidates.
        best_word (str): The best candidate word according to Norvig's solution.
        previous_word (str): The previous word in the sequence. Default is None.

    Returns:
        tuple: A tuple containing the selected word and possible next words.
    """
    candidates = calculate(current_word, next_word, best_word, previous_word=previous_word)
    # Sort candidates by cost in descending order and select the top one
    candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[0]
    return candidates[0], candidates[2]

In [12]:
def last_word(previous_word, current_word):
    """
    Determine the last word based on the previous word and current word candidates.

    Parameters:
        previous_word (str): The previous word in the sequence.
        current_word (list): List of current word candidates.

    Returns:
        str: The selected last word.
    """
    max_k = 0
    max_word = ''
    # Find the word with the highest bigram probability
    for word in current_word:
        if (previous_word, word) in bigram_dict:
            k = bigram_dict[(previous_word, word)]
            if k > max_k:
                max_k = k
                max_word = word
    # If no word found based on bigram probability, select the word with the highest unigram probability
    if max_k == 0:
        for word in current_word:
            if (previous_word, word) in bigram_dict:
                k = vocab_check_dict[(' ', word)]
                if k > max_k:
                    max_k = k
                    max_word = word
    # If still no word found, select the first word in the list of candidates
    if max_k == 0:
        max_word = current_word[0]
    return max_word

In [13]:
def spelling_corrector_ngram(sentence):
    """
    Correct spelling in a sentence using an bigram based approach.

    Parameters:
        sentence (str): The input sentence.

    Returns:
        str: The corrected sentence.
    """
    sentence_arr = sentence.lower().split()
    new_sentence = []
    for i in range(len(sentence_arr) - 1):
        if i == 0:
            # Determine alternatives or variants for the first word
            if (sentence_arr[i], ' ') not in vocab_check:
                new_sentence.append(variants_word(sentence_arr[i], 1))
            else:
                new_sentence.append(alternatives_word(sentence_arr[i], 1))
        # Determine alternatives or variants for the next word
        if (' ', sentence_arr[i + 1]) not in vocab_check:
            new_sentence.append(variants_word(sentence_arr[i + 1], 2))
        else:
            new_sentence.append(alternatives_word(sentence_arr[i + 1], 2))
        # Find the best word based on the context
        if i != 0:
            new_sentence[i], new_sentence[i + 1] = find_word(new_sentence[i], new_sentence[i + 1], correction(sentence_arr[i]), new_sentence[i - 1])
        else:
            new_sentence[i], new_sentence[i + 1] = find_word(new_sentence[i], new_sentence[i + 1], correction(sentence_arr[i]))

        # Find the last word based on the previous word
        if i == len(sentence_arr) - 2:
            new_sentence[i+1] = last_word(new_sentence[i], new_sentence[i + 1])

    return ' '.join(new_sentence)

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

My solution is built on a modified bigram. I consider not only context between two words but also look at previous word to ensure that the word fits the context. This can help where the previous word strongly influences the choice of the current word. Also, Instead of just considering pairs of words, I look how one word combines with different variations of the next word. I looking for options based on the frequency of word combinations and their fitness with the previous context. This approach allows me to consider that the chosen word not only occurs frequently, but also fits well into the sentence context.

For example,

'I wnnt o neet with fried' (I want to meet with friends)

1. ['i', 'it', 'is'] + ['want', 'went'] - possible first and second words
2. ('i', 'want') + ('i', 'went') = 8000, ('it', 'want') + ('it', 'went') = 700, ('is', 'want') + ('is', 'went') = 0 - based on the following variations, which word would be better fit
3. 'i' + ['want', 'went'] + ['of', 'to', 'on'] - put the most frequently used word, and now the next word is being considered
4. ('i', 'want') + ('want', 'of') + ... + ('want', 'on') = 3000, ('i', 'went') + ('went', 'of') + ... + ('went', 'on') = 2000 - in all words except the first, expect that our current word should match the previous one
5. etc.

I provide incentives for words that fit well with the previous context and also consider the word suggested by Norvig's solution, potentially giving it higher priority.

My solution differs from Norvig's by considering alternative words even if the current word exists. This flexibility allows for better adaptation to the context and potentially improves correction accuracy (function: alternative_word()).
But my solution uses some functions from Norvig's solution to quickly find the correct words.







## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [14]:
import random

In [15]:
# sentences are randomly taken from https://www.memorysecrets.ru/english/en-texts.html
sentences = ['Travelling to far countries is always a thrilling and interesting adventure',
             'On vacation you forget the hustle and bustle of everyday life and your job',
             'Vegetables have a tiny amount of calories and are very rich in fiber',
             'Absolutely all people of every type around the world are not indifferent to the music.',
             'A week ago I got sick with the simple cold and did not want to go to the doctor.',
             'Choosing a future profession influences all the future life of a person',
             'All of us remember their first books from childhood',
             'Earlier the explorers discovered deserted islands with the beaches of fine sand.',
             'Our planet is the only place where a human being might live',
             'All girls and boys are required to wear a particular uniform.']

In [17]:
test_dataset = []
for sentence in sentences:
    for _ in range(5):
        new_sentence = []
        for word in sentence.split():
            # Introduce spelling errors with a probability of 80%
            if random.random() > 0.2:
                # Decide whether to make a simple edit (40%) or a more complex edit (60%)
                if random.random() > 0.6:
                    v = list(edits1(word))
                else:
                    v = list(edits2(word))
                new_sentence.append(random.choice(v))  # Randomly select an edit from the list
            else:
                new_sentence.append(word)
        test_dataset.append((' '.join(new_sentence), sentence.lower()))

test_dataset[4:9]

[('Tsravelling rto fari countrxbes his alwfns fwa thrifuling and pnterestjing aduventure',
  'travelling to far countries is always a thrilling and interesting adventure'),
 ('dObn vacativon yizou bforget tjhf huztle anud blsktle ef everyday life jsnd yeodr jrob',
  'on vacation you forget the hustle and bustle of everyday life and your job'),
 ('Ojn valcaion you forgetwj tje wustle awd bustlaa cpf everyady life annd yourq jonsb',
  'on vacation you forget the hustle and bustle of everyday life and your job'),
 ('On tvacatizn yen fqorget the hustqle dnsd beustlbe bojf everyday liqe gnd you jdoh',
  'on vacation you forget the hustle and bustle of everyday life and your job'),
 ('Oonc nvacation kaou forgqt tvhe hutstle and buwstlei foo severyday life aedo youm jhb',
  'on vacation you forget the hustle and bustle of everyday life and your job')]

In [18]:
# from https://stackoverflow.com/questions/17388213/find-the-similarity-metric-between-two-strings
from difflib import SequenceMatcher

def similar(pred, target):
    return SequenceMatcher(None, pred, target).ratio()

In [20]:
ngram_similar = []
norvig_similar = []
for sentence in test_dataset:

    print('A sentence with an error:', sentence[0])
    ngram = spelling_corrector_ngram(sentence[0])
    norvig = spelling_corrector_norvig(sentence[0])


    ngram_similar.append(similar(ngram, sentence[1]))
    norvig_similar.append(similar(norvig, sentence[1]))


    print('A sentence corrected using Ngram:\n', ngram, '\nScore (ngram):', ngram_similar[-1])
    print('A sentence corrected using Norvig:\n', norvig, '\nScore (norvig):', norvig_similar[-1])
    print('A correct sentence:', sentence[1] + '\n')


A sentence with an error: Tratevlling ho acar csuntiries gsw aalvays oao thrilling anqy interesytcng asventure
A sentence corrected using Ngram:
 travelling to scar countries is always oak thrilling any interesting adventure 
Score (ngram): 0.954248366013072
A sentence corrected using Norvig:
 travelling ho scar countries is always oak thrilling any interesting adventure 
Score (norvig): 0.9411764705882353
A correct sentence: travelling to far countries is always a thrilling and interesting adventure

A sentence with an error: Traveleidg nto far countries oixs ahsways nv rthrildling and tnteresting advewnturx
A sentence corrected using Ngram:
 traveled to war countries oils always no thrilling and interesting adventure 
Score (ngram): 0.9139072847682119
A sentence corrected using Norvig:
 traveling to far countries oils always no thrilling and interesting adventure 
Score (norvig): 0.9605263157894737
A correct sentence: travelling to far countries is always a thrilling and interesting 

In [25]:
acc_ngram = sum(ngram_similar) / len(ngram_similar)
acc_ngram

0.9575582845646727

In [26]:
acc_norvig = sum(norvig_similar) / len(norvig_similar)
acc_norvig

0.93035239979529

Since the sentences are not very complex and there are not many errors, both algorithms work well, but since my solution takes context into account, it produces better results with different generated sentences.